In [3]:
!pip install transformers


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import math

def calculate_perplexity(text, model_name="gpt2"):
    # Load tokenizer and model
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    
    # Tokenize input text and move to tensor
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    input_ids = inputs.input_ids
    
    # Disable gradient computation
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
        loss = outputs.loss  # Cross-entropy loss

    # Compute perplexity
    perplexity = math.exp(loss.item())
    return perplexity

# Example usage
text = "The quick brown fox jumps over the lazy dog."
ppl = calculate_perplexity(text)
print(f"Perplexity: {ppl:.2f}")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Perplexity: 162.47


In [3]:
import textstat

def calculate_readability(text):
    # Flesch-Kincaid Readability Score
    flesch_score = textstat.flesch_reading_ease(text)
    
    # Gunning Fog Index
    fog_index = textstat.gunning_fog(text)
    
    return {
        "Flesch Reading Ease": flesch_score,
        "Gunning Fog Index": fog_index
    }

# Example usage
text = "The quick brown fox jumps over the lazy dog. This sentence is simple and easy to read."
readability_scores = calculate_readability(text)
print(readability_scores)


{'Flesch Reading Ease': 88.23, 'Gunning Fog Index': 3.4}


In [17]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load RoBERTa trained on MNLI
model_name = "roberta-large-mnli"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Function to check entailment
def check_entailment(premise, hypothesis):
    """
    Determines if the hypothesis (generated response) logically follows from the premise (previous conversation).
    """
    inputs = tokenizer(premise, hypothesis, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        logits = model(**inputs).logits

    # Softmax to get probabilities
    probs = torch.nn.functional.softmax(logits, dim=-1)
    labels = ["contradiction", "neutral", "entailment"]

    # Get highest probability label
    prediction = labels[torch.argmax(probs).item()]
    
    return {
        "entailment_score": probs[0][2].item(),
        "neutral_score": probs[0][1].item(),
        "contradiction_score": probs[0][0].item(),
        "prediction": prediction
    }

# Example usage
premise = "It's so sunny outside"
hypothesis = "The sky is totally full of clouds"

result = check_entailment(premise, hypothesis)
print(result)

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'entailment_score': 0.0002006716385949403, 'neutral_score': 0.0008906660368666053, 'contradiction_score': 0.9989086389541626, 'prediction': 'contradiction'}


In [18]:
import allennlp_models
from allennlp.predictors.predictor import Predictor

# Load the coreference resolution model
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2021.03.10.tar.gz")

def resolve_coreferences(text):
    """
    Resolves coreferences in the input text and returns a version with resolved entities.
    """
    result = predictor.predict(document=text)

    # Get clusters of coreferent mentions
    clusters = result["clusters"]
    words = result["document"]
    
    # Map coreferences to their main entity
    coref_map = {}
    for cluster in clusters:
        main_entity = " ".join(words[cluster[0][0] : cluster[0][1] + 1])  # First mention
        for mention in cluster[1:]:  # Replace other mentions
            coref_map[" ".join(words[mention[0] : mention[1] + 1])] = main_entity

    # Replace coreferent mentions in the text
    resolved_text = []
    for word in words:
        resolved_text.append(coref_map.get(word, word))

    return " ".join(resolved_text)

# Example usage
text = "Alice said she would go to the store. Later, he bought some apples."
resolved_text = resolve_coreferences(text)
print(resolved_text)


ModuleNotFoundError: No module named 'allennlp_models'

In [26]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load a DeBERTa model fine-tuned on EmpatheticDialogues
model_name = "microsoft/deberta-v3-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

def measure_empathy(context, response):
    """
    Uses a fine-tuned model to assess how empathetic the response is.
    Returns an empathy score.
    """
    inputs = tokenizer(context, response, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        logits = model(**inputs).logits

    # Softmax to get probabilities
    probs = torch.nn.functional.softmax(logits, dim=-1)
    
    print(probs)

    return {
        "low_empathy_score": probs[0][0].item(),
        "medium_empathy_score": probs[0][1].item(),
        "high_empathy_score": probs[0][2].item()
    }

# Example usage
context = "I had an accident. I can't walk."
response = "Okay."

result = measure_empathy(context, response)
print(result)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([[0.4136, 0.5864]])


IndexError: index 2 is out of bounds for dimension 0 with size 2

In [63]:
import torch
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from typing import Dict, Tuple

class EmpathicDialogueEvaluator:
    def __init__(self):
        self._load_models()
        
    def _load_models(self):
        """Load all required pretrained models"""
        # Empathic Dialogue Model (using a fine-tuned T5 for dialogue)
        self.edm = pipeline(
            "text2text-generation",
            model="microsoft/DialoGPT-medium",
            device=0 if torch.cuda.is_available() else -1
        )
        
        # Emotion classification model (BERT-based)
        self.emotion_model_name = "bert-base-uncased"
        self.emotion_tokenizer = AutoTokenizer.from_pretrained(self.emotion_model_name)
        self.emotion_model = AutoModelForSequenceClassification.from_pretrained(
            "bhadresh-savani/bert-base-uncased-emotion",
            num_labels=6
        )
        self.emotion_model.eval()
    
    def detect_emotion(self, text: str) -> Dict[str, float]:
        """Detect emotions in text"""
        inputs = self.emotion_tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            padding=True
        )
        
        with torch.no_grad():
            outputs = self.emotion_model(**inputs)
        
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1).squeeze()
        
        emotion_labels = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
        emotion_scores = {label: float(prob) for label, prob in zip(emotion_labels, probabilities)}
        
        return emotion_scores
    
    def dot_product(self, dict1, dict2):
        return sum(dict1[k] * dict2[k] for k in dict1 if k in dict2)
    
    def evaluate_empathy(self, context: str, response: str) -> Dict[str, float]:
        """Evaluate how empathic a response is to the given context"""
        # Get context emotion
        context_emotion = self.detect_emotion(context)
        primary_context_emotion = max(context_emotion.items(), key=lambda x: x[1])[0]
        
        # Get response emotion
        response_emotion = self.detect_emotion(response)
        print(context_emotion, response_emotion)
        
        emotion_alignment = response_emotion.get(primary_context_emotion, 0.0)
        # print(emotion_alignment)
        print(self.dot_product(context_emotion, response_emotion))
        print(context_emotion[primary_context_emotion] * response_emotion[primary_context_emotion])
        
        return {
            # 'empathy_score': empathy_score,
            'emotion_alignment': emotion_alignment,
            'context_emotion': primary_context_emotion,
            'response_emotion_distribution': response_emotion
        }
    
    def full_pipeline(self, context: str, response: str) -> Tuple[str, Dict[str, float]]:
        """Complete pipeline from context to evaluated response"""
        evaluation = self.evaluate_empathy(context, response)
        return evaluation


def empathy(context, response):
    evaluator = EmpathicDialogueEvaluator()
    
    # Generate and evaluate response
    evaluation = evaluator.full_pipeline(context, response)
    
    # print(f"Context: {context}")
    # print(f"Generated Response: {response}")
    # print("\nEvaluation Metrics:")
    # # print(f"Empathy Score: {evaluation['empathy_score']:.3f}")
    # print(f"Emotion Alignment: {evaluation['emotion_alignment']:.3f}")
    # print(f"Context Emotion: {evaluation['context_emotion']}")
    # print("Response Emotion Distribution:")
    # for emotion, score in evaluation['response_emotion_distribution'].items():
    #     print(f"  {emotion}: {score:.3f}")
    
    
context = "I had an accident. I can't walk."
response = "I am so sorry to hear that. is there anything I can do to help you?"
empathy(context, response)

Device set to use cpu
The model 'GPT2LMHeadModel' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'Qwen2AudioForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration']

{'sadness': 0.3279312551021576, 'joy': 0.0019324220484122634, 'love': 0.0007190885371528566, 'anger': 0.006600219290703535, 'fear': 0.659487783908844, 'surprise': 0.003329205559566617} {'sadness': 0.9626277685165405, 'joy': 0.01635679043829441, 'love': 0.0037591566797345877, 'anger': 0.00739735271781683, 'fear': 0.009084840305149555, 'surprise': 0.0007740959408693016}
0.3217527861894074
0.005991341200008826


In [64]:
context = "I had an accident. I can't walk."
response = "I am so sorry to hear that. is there anything I can do to help you?"
empathy(context, response)

context = "I had an accident. I can't walk."
response = "Okay."
empathy(context, response)

context = "I had an accident. I can't walk."
response = "Haha. So funny."
empathy(context, response)


Device set to use cpu
The model 'GPT2LMHeadModel' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'Qwen2AudioForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration']

{'sadness': 0.3279312551021576, 'joy': 0.0019324220484122634, 'love': 0.0007190885371528566, 'anger': 0.006600219290703535, 'fear': 0.659487783908844, 'surprise': 0.003329205559566617} {'sadness': 0.9626277685165405, 'joy': 0.01635679043829441, 'love': 0.0037591566797345877, 'anger': 0.00739735271781683, 'fear': 0.009084840305149555, 'surprise': 0.0007740959408693016}
0.3217527861894074
0.005991341200008826


Device set to use cpu
The model 'GPT2LMHeadModel' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'Qwen2AudioForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration']

{'sadness': 0.3279312551021576, 'joy': 0.0019324220484122634, 'love': 0.0007190885371528566, 'anger': 0.006600219290703535, 'fear': 0.659487783908844, 'surprise': 0.003329205559566617} {'sadness': 0.004103075247257948, 'joy': 0.9866077899932861, 'love': 0.002547743497416377, 'anger': 0.002644197316840291, 'fear': 0.002923656953498721, 'surprise': 0.0011735305888578296}
0.005203376567147151
0.0019281160451725537


Device set to use cpu
The model 'GPT2LMHeadModel' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'Qwen2AudioForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration']

{'sadness': 0.3279312551021576, 'joy': 0.0019324220484122634, 'love': 0.0007190885371528566, 'anger': 0.006600219290703535, 'fear': 0.659487783908844, 'surprise': 0.003329205559566617} {'sadness': 0.0009785917354747653, 'joy': 0.011624733917415142, 'love': 0.007527983281761408, 'anger': 0.00934939831495285, 'fear': 0.0016754867974668741, 'surprise': 0.9688438177108765}
0.004740939373081144
0.001104963075029955


In [66]:
context = "I just won the world championship brother!"
response = "Wow! I am so happy for you. We should celebrate."
empathy(context, response)

context = "I just won the world championship brother!"
response = "Ok."
empathy(context, response)

context = "I just won the world championship brother!"
response = "Ugh! That sucks. Who cares man?"
empathy(context, response)

Device set to use cpu
The model 'GPT2LMHeadModel' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'Qwen2AudioForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration']

{'sadness': 0.0006771081243641675, 'joy': 0.995418906211853, 'love': 0.0009584332001395524, 'anger': 0.0022520895581692457, 'fear': 0.0002604444161988795, 'surprise': 0.0004329569637775421} {'sadness': 0.00043299945537000895, 'joy': 0.9985278844833374, 'love': 0.000645835476461798, 'anger': 0.00012815202353522182, 'fear': 8.399751095566899e-05, 'surprise': 0.00018109528173226863}
0.9939548356650308
0.9939535345944392


Device set to use cpu
The model 'GPT2LMHeadModel' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'Qwen2AudioForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration']

{'sadness': 0.0006771081243641675, 'joy': 0.995418906211853, 'love': 0.0009584332001395524, 'anger': 0.0022520895581692457, 'fear': 0.0002604444161988795, 'surprise': 0.0004329569637775421} {'sadness': 0.005043392535299063, 'joy': 0.9863716959953308, 'love': 0.0017419852083548903, 'anger': 0.0034630002919584513, 'fear': 0.002347860485315323, 'surprise': 0.0010320197325199842}
0.9818669765386012
0.9818530347460026


Device set to use cpu
The model 'GPT2LMHeadModel' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'Qwen2AudioForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration']

{'sadness': 0.0006771081243641675, 'joy': 0.995418906211853, 'love': 0.0009584332001395524, 'anger': 0.0022520895581692457, 'fear': 0.0002604444161988795, 'surprise': 0.0004329569637775421} {'sadness': 0.0558466874063015, 'joy': 0.08871188014745712, 'love': 0.027992727234959602, 'anger': 0.8151504993438721, 'fear': 0.007115386892110109, 'surprise': 0.005182795226573944}
0.09021001512726394
0.08830548270437877


In [74]:
from nrclex import NRCLex
import nltk
nltk.download('punkt_tab')

def get_emotion_scores(text):
    """Returns emotion intensity scores from NRC Lexicon."""
    emotion = NRCLex(text)
    return emotion.affect_frequencies

def check_emotion_alignment(user_input, response):
    """Compares the emotional intensity of user input and response."""
    user_emotions = get_emotion_scores(user_input)
    response_emotions = get_emotion_scores(response)

    # Example: Ensure sadness in input is not countered by excessive joy in response
    if user_emotions['sadness'] > 0.3 and response_emotions['joy'] > 0.5:
        aligned = False  # Response is overly positive
    else:
        aligned = True

    return {
        "user_emotions": user_emotions,
        "response_emotions": response_emotions,
        "aligned": aligned
    }

# Example
user_input = "I'm feeling really down today."
response = "Don't worry, be happy! Everything is great!"

print(check_emotion_alignment(user_input, response))


[nltk_data] Error loading punkt_tab: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


MissingCorpusError: 
Looks like you are missing some required data for this feature.

To download the necessary data, simply run

    python -m textblob.download_corpora

or use the NLTK downloader to download the missing data: http://nltk.org/data.html
If this doesn't fix the problem, file an issue at https://github.com/sloria/TextBlob/issues.


In [75]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load Empathy Fine-Tuned Alpaca Model (if available on Hugging Face)
model_name = "tatsu-lab/alpaca-7b"  # Replace with the fine-tuned version if available

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    output = model.generate(**inputs, max_length=150)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Example Input
user_input = "I'm feeling really lost and hopeless today."
response = generate_response(user_input)
print("Model Response:", response)


tokenizer_config.json:   0%|          | 0.00/836 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggin

...

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/7.18G [00:00<?, ?B/s]

In [10]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModel, AutoModelForMaskedLM
import torch
import numpy as np

# Load PsychBERT or MentalBERT (Fine-tuned on mental health data)
model_name = "mnaylor/psychbert-cased"  # or "yiyanghkust/bert-psychology"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name, from_flax=True)

def classify_empathy(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    scores = torch.nn.functional.softmax(outputs.logits, dim=-1)
    return scores.detach().numpy()

# Example Input
response_text = "I understand that you're feeling lost. I'm here to listen and support you."
empathy_score = classify_empathy(response_text)
final_empathy_score = np.max(empathy_score)
print("Empathy Score:", final_empathy_score)


All Flax model weights were used when initializing BertForMaskedLM.

Some weights of BertForMaskedLM were not initialized from the Flax model and are newly initialized: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Empathy Score: 0.99999964


In [14]:
response_text1 = "I had an accident today. probably I can't walk anymore"
empathy_score1 = classify_empathy(response_text)

response_text2 = "haha. That's so funny."
empathy_score2 = classify_empathy(response_text)

vec1 = empathy_score1.flatten() 
vec2 = empathy_score2.flatten()

dot_product = np.dot(vec1, vec2)
print("Empathy Match Score:", dot_product)

ValueError: shapes (1,15,28996) and (1,15,28996) not aligned: 28996 (dim 2) != 15 (dim 1)